In [233]:
import time
import numpy as np
import pandas as pd

from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


# Controlling the webdriver  
Firstly, we want to access the webpage doctolib. To do that we have to create a connector with Selenium and the launch the driver

In [234]:
tempsAttente = 3    #temps d'attente en secondes avant qu'il y a un timeout de Webdriverwait.until()

class Driver:

    def __init__(self):
        options = webdriver.ChromeOptions()
        options.add_argument("start-maximized")
        options.add_argument("--disable-blink-features")
        options.add_argument("disable-popup-blocking")
        options.add_argument("disable-notifications")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        self.driver = webdriver.Chrome(ChromeDriverManager().install())


    #Lancer google chrome avec la page doctolib

    def connect(self, url):
        self.driver.get(url)
        # TabsInteraction.TabsCreation(self.driver, "https://www.doctolib.fr", "")
        # TabsInteraction.switchToTab(self.driver, 0)
        print("Connecting to", url)
        time.sleep(1)


    #for now we get only the first doctors that appear in the page
    def getDoctors(self, specialization, city):

        doctors = []

        #writing the specialization
        WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]')))
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]').clear()
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]').send_keys(specialization)
        time.sleep(1)

        #writing the city
        WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-place-input"]')))
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-place-input"]').clear()
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-place-input"]').send_keys(city)
        time.sleep(1)
        #We need to enter the city to register it
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-place-input"]').send_keys(Keys.ENTER)

        WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//span[@class="searchbar-submit-button-label dl-text-transform-none"]')))
        self.driver.find_element(By.XPATH, '//span[@class="searchbar-submit-button-label dl-text-transform-none"]').click()

        print('Searching results for :', specialization, 'in', city)
        time.sleep(2)

        WebDriverWait(self.driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//h4[@class="dl-text dl-text-body dl-text-bold dl-text-s dl-text-neutral-150 total-number-of-results dl-margin-r-8"]')))
        #number of doctors in city
        nbr_result = self.driver.find_element(By.XPATH, '//h4[@class="dl-text dl-text-body dl-text-bold dl-text-s dl-text-neutral-150 total-number-of-results dl-margin-r-8"]').text.split(" ")[0]
        #nbr of doctors we want
        nbr = int(int(nbr_result)*0.2)
        #nbr of pages
        pages = nbr//20
        print(pages)

        for _ in range(pages + 1):
            
            self.driver.execute_script("window.scrollTo(0, 6300);")
            
            WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="dl-search-result"]')))
            docs = self.driver.find_elements(By.XPATH, '//div[@class="dl-search-result"]')
            
            for doctor in docs:
                #first we get the "easy" features
                name = doctor.find_element(By.XPATH, './/h3[@data-test="dl-search-result-name"]').text
                city = city
                specialization = specialization

                #the the harder ones
                try:
                    WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, './/div[@class="Tappable-inactive availabilities-slot"]')))
                    first_dispo = doctor.find_element(By.XPATH, './/div[@class="Tappable-inactive availabilities-slot"]').get_attribute('aria-label')
                except:
                    try:
                        WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, './/span[@class="dl-button-label"]')))
                        first_dispo = doctor.find_elements(By.XPATH, './/span[@class="dl-button-label"]')[1].text.split(" ")[3:5]
                    except:
                        first_dispo = 'null'    #means that there are no near in time disponibilites

                print(name, first_dispo)
                fee = ""
                doctors.append(Doctor(name, city, specialization, first_dispo, fee))

            try:
                WebDriverWait(self.driver, 3).until(EC.visibility_of_element_located((By.XPATH, "//a[text()='Suivant']")))
                self.driver.find_element(By.XPATH, "//a[text()='Suivant']").click()
            except:
                pass


        return doctors

    def getFees(self, name):
        self.driver.find_element(By.XPATH, '//div[@class="logo-doctolib logo-doctolib-white"]').click()

        #writing the doctor name
        WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]')))
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]').clear()
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]').send_keys(name)
        #We need to enter the city to register it

        try:
            WebDriverWait(self.driver, 0.5).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-query-input searchbar-query-input-opened"]')))
            self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input searchbar-query-input-opened"]').send_keys(Keys.ENTER)
        except:
            return 'Weird value'

        try:
            WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="dl-profile-fee-tag"]')))
            fee = self.driver.find_element(By.XPATH, '//div[@class="dl-profile-fee-tag"]').text
            fee = str(int(fee.split(" ")[0]))
        except:
            fee = 'Conventionné'        #if the fee tag cannot be found it means that it is conventionné

        return fee
            
        


We created a Doctor class to manipulate more easily the doctors features

In [235]:
class Doctor():
    def __init__(self, name, city, specialization, first_dispo, fee):
        self.name = name
        self.city = city
        self.specialization = specialization
        self.first_dispo = first_dispo
        self.fee = fee

    def features(self):
        return(self.name, self.city, self.specialization, self.first_dispo, self.fee)

These are the cities and specializations we looked at first :  
Lyon and Paris are bug cities and Sancerre and Confolens are small ones, we did this in order to compare fees and disponibilites at first glance.

In [236]:
cities = ['Poitiers', 'Rochefourchat', 'Majastres', 'Aulan', 'Ornes', 'Baren', 'Senconac', 'Caubous', 'Sancerre', 'Paris', 'Lyon', 'Marseille', 'Toulouse', 'Nice', 'Nantes', 'Montpellier', 'Strasbourg', 'Bordeaux', 'Lille', 'Poitiers', 'Dunkerque', 'Pau', 'Mulhouse', 'Lorient', 'Le Havre', 'Annecy', 'Troyes', 'La Rochelle', 'Amiens']
specializations = ['Medecin generaliste', 'Chirurgien-dentiste', 'Ophtalmologue', 'Dermatologue et venerologue', 'Cardiologue', 'Rhumatologue']

In [237]:
connector = Driver()
connector.connect("https://www.doctolib.fr")
doctors = []

#Get all the doctors names, cities, specializations and their next possible reservation

for city in cities:
    for spec in specializations:
        doctors = doctors + connector.getDoctors(spec, city)
        docs = []
        for doc in doctors:
            docs.append(doc.features())
        pd.DataFrame(docs, columns=['Name', 'Specialization', 'City', 'Waiting Time', 'Fees']).to_csv("doctors.csv")

/var/folders/bm/t8hxv1ms6s30y8czfw0pmfk00000gn/T/ipykernel_2019/1043591837.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(ChromeDriverManager().install())


Connecting to https://www.doctolib.fr
Searching results for : Medecin generaliste in Poitiers
1
Dr David CHAMBENOIT ['18', 'janvier']
Dr Céline BRECHON-GIRAUD ['11', 'janvier']
Dr Alain SHEIKHALISHAHI ['26', 'janvier']
Dr Violaine Ricoulleau ['11', 'janvier']
Dr Airelle PLANQUES ['16', 'janvier']
Laura BORTHOMIEU ['18', 'janvier']
Dr Angèle DEMION ['12', 'janvier']
Dr Karine Brunet ['17', 'janvier']
Dr Céline BRECHON-GIRAUD ['12', 'janvier']
Centre Médical de Soins Immédiats 86 jeu. 5 janv. 17:00
Dr Cécile HUMMEL ['9', 'janvier']
Dr Elsa CANTOS jeu. 5 janv. 16:30
Dr Claire REIF null
Dr Sébastien Caquelin null
Patricia Mercade Point null
Dr Mathilde COULAIS null
Stephane Colin null
Dr Lakshmipriya Le Bonheur null
Dr François Birault null
Dr Gabrielle Pequeriau null
Dr Lucile Partaud null
Dr Marion Delouche null
Dr David Souchaud null
Dr Stéphanie Mignot - Grandcolin null
Dr Laurent Toma null
Nathalie Dumureau Peltier null
Marc BAHUET null
Christophe INGRAND null
Dr Déborah CARRIN null
D

In [ ]:
for doctor in doctors:
    if doctor.city == 'Paris':
        print(doctor.features())

('Dr Joël Valendoff', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Ridha MAHJOUB', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Léa Athias', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Patricia Mussuto', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Maurice KONQUI', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Didier Montloin', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Isaac Bohbot', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Serge ALPERIN', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Françoise BARRO-LECOMTE', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Eric FISCHER', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Jean-Luc THOMAS', 'Paris', 'Medecin generaliste', 'null', '')
('Institut Alfred Fournier', 'Paris', 'Medecin generaliste', 'null', '')
('Centre de Santé (CDS) - Bauchat Nation', 'Paris', 'Medecin generaliste', 'null', '')
('Centre de santé de Belleville', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Marie Paule VIGLA', 'Paris',

We can see that there are no disponibilites for any doctor. This means that we will have to dig for more doctors as we are only taking the first ones at the moment.

In [ ]:
#Now we need to find the fees for each doctor

connector = Driver()
connector.connect("https://www.doctolib.fr")

for doctor in doctors:
    fee = connector.getFees(doctor.name)
    doctor.fee = fee
    

In [ ]:
doctors_features = []

for doctor in doctors:
    doctors_features.append(doctor.features())

In [ ]:
pd.DataFrame(doctors_features, columns=['Name', 'Specialization', 'City', 'Waiting Time', 'Fees']).to_csv("doctors_features.csv")

In [ ]:
print(doctors)

[<__main__.Doctor object at 0x1441331d0>, <__main__.Doctor object at 0x14566acd0>, <__main__.Doctor object at 0x14572f9d0>, <__main__.Doctor object at 0x1457ce690>, <__main__.Doctor object at 0x145578310>, <__main__.Doctor object at 0x1457cf710>, <__main__.Doctor object at 0x1457cfbd0>, <__main__.Doctor object at 0x1457cc790>, <__main__.Doctor object at 0x1457cd110>, <__main__.Doctor object at 0x14566b190>, <__main__.Doctor object at 0x1457cc390>, <__main__.Doctor object at 0x14566bb90>, <__main__.Doctor object at 0x144132850>, <__main__.Doctor object at 0x1457cff10>, <__main__.Doctor object at 0x14566af50>, <__main__.Doctor object at 0x1457cc8d0>, <__main__.Doctor object at 0x1457bc050>, <__main__.Doctor object at 0x14572da90>, <__main__.Doctor object at 0x14566a450>, <__main__.Doctor object at 0x1457bd7d0>, <__main__.Doctor object at 0x14566a510>, <__main__.Doctor object at 0x1457ce950>, <__main__.Doctor object at 0x14572c690>, <__main__.Doctor object at 0x14412e310>, <__main__.Docto